In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
%%cuda

UsageError: %%cuda is a cell magic, but the cell body is empty.


In [ ]:
%%writefile reduction_v1.cu
// Naive interleaved reduction kernel
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void reduce_v1(float *input, float * output, int n){
    extern __shared__ float sdata[];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // load two elements per thread
    float sum = 0.0f;
    if (idx < n){
        sum = input[idx];
        if (idx + blockDim.x < n){
            sum += input[idx + blockDim.x];
        }
    }

    sdata[tid] = sum;
    __syncthreads();

    // interleaved reduction . Stride halves every interation
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1){
        if (tid < stride){
            sdata[tid] += sdata[tid + stride];
        }
        __syncthreads();
    }
    if (tid == 0){
        output[blockIdx.x] = sdata[0];
    }
}

int main(){
    int N = 1 << 20; // 1M elements
    size_t size = N * sizeof(float);

    const int blockSizes[] = {128, 256, 512};
    const int numTests = 3;

    float *h_input = (float *)malloc(size);

    for (int i=0; i< N; i++){
        h_input[i] = 1.0f; // Initialize all elements to 1.0f
    }

    float *d_input, *d_output;
    cudaMalloc((void **)&d_input, size);

    for (int t=0; t< numTests; t++){
        int threadsPerBlock = blockSizes[t];
        int blocks = (N + threadsPerBlock * 2 - 1) / (threadsPerBlock * 2);

        cudaMalloc((void **)&d_output, blocks * sizeof(float));
        cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);

        printf("\nTesting block size: %d (blocks: %d)\n", threadsPerBlock, blocks);

        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        cudaEventRecord(start);
        reduce_v1<<<blocks, threadsPerBlock, threadsPerBlock * sizeof(float)>>>(d_input, d_output, N);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess) {
            fprintf(stderr, "CUDA error: %s\n", cudaGetErrorString(err));
            return -1;
        }

        float gpu_time = 0;
        cudaEventElapsedTime(&gpu_time, start, stop);
        printf("Time taken for v1: %.2f ms\n", gpu_time);

        float *h_output = (float *)malloc(blocks * sizeof(float));
        cudaMemcpy(h_output, d_output, blocks * sizeof(float), cudaMemcpyDeviceToHost);

        float final_sum = 0.0f;
        for (int i = 0; i < blocks; i++) {
            final_sum += h_output[i];
        }

        printf("Final Sum = %.0f (expected %d)\n", final_sum, N);
        free(h_output);
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    return 0;

}

Writing reduction_v1.cu


In [ ]:
!nvcc -arch=sm_75 reduction_v1.cu -o reduction_v1

In [ ]:
!./reduction_v1


Testing block size: 128 (blocks: 4096)
Time taken for v1: 0.24 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 256 (blocks: 2048)
Time taken for v1: 0.07 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 512 (blocks: 1024)
Time taken for v1: 0.07 ms
Final Sum = 1048576 (expected 1048576)


In [ ]:
%%writefile reduction_v2.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void reduce_v2(float *i, float *o, int n){
    extern __shared__ float std[];

    int tid = threadIdx.x; // thread index within the block
    int idx  = blockIdx.x * (blockDim.x * 2) + threadIdx.x;

    // load data into shared memory
    float sum = 0.0f;
    if (idx < n){
        sum = i[idx];
        if (idx + blockDim.x < n){
            sum += i[idx + blockDim.x];
        }
    }

    // store the partial sum in shared memory
    std[tid] = sum;
    __syncthreads();

    // perform reduction in shared memory
    for (int stride=1; stride < blockDim.x; stride <<=1){
        int index = 2 * stride * tid;
        if (index < blockDim.x){
            std[index] += std[index + stride];
        }
        __syncthreads();
    }

    if (tid == 0){
        o[blockIdx.x] = std[0];
    }
}

int main(){
    int N = 1 << 20; // 1M elements
    size_t size = N * sizeof(float);

    const int blockSizes[] = {128, 256, 512};
    const int numTests = 3;

    // Allocate host memory
    float *h_i = (float *)malloc(size);
    // Initialize input data
    for (int i=0; i<N; i++){
        h_i[i] = 1.0f;
    }

    float *d_a, *d_b;
    cudaMalloc((void **)&d_a, size);

    for(int t=0; t<numTests; t++){
        int threadsPerBlock = blockSizes[t];
        int blocks = (N + threadsPerBlock * 2 - 1) / (threadsPerBlock * 2);

        cudaMalloc(&d_b, blocks * sizeof(float));
        cudaMemcpy(d_a, h_i, size, cudaMemcpyHostToDevice);

        printf("\nTesting block size: %d (blocks: %d)\n", threadsPerBlock, blocks);

        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        cudaEventRecord(start);
        reduce_v2<<<blocks, threadsPerBlock, threadsPerBlock * sizeof(float)>>>(d_a, d_b, N);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess){
            printf("CUDA Error: %s\n", cudaGetErrorString(err));
            return -1;
        }

        float gpu_time = 0.0f;
        cudaEventElapsedTime(&gpu_time, start, stop);
        printf("Time taken for v2: %.4f ms\n", gpu_time);

        float *h_o = (float *)malloc(blocks * sizeof(float));
        cudaMemcpy(h_o, d_b, blocks * sizeof(float), cudaMemcpyDeviceToHost);

        float final_sum = 0;
        for(int i=0; i<blocks; i++){
            final_sum += h_o[i];
        }
        printf("Final Sum = %.0f (expected %d)\n", final_sum, N);

        free(h_o);
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
        cudaFree(d_b);
    }
    cudaFree(d_a);
    free(h_i);
    return 0;
}

Writing reduction_v2.cu


In [ ]:
!nvcc -arch=sm_75 reduction_v2.cu -o reduction_v2

In [ ]:
!./reduction_v2


Testing block size: 128 (blocks: 4096)
Time taken for v2: 0.1401 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 256 (blocks: 2048)
Time taken for v2: 0.0783 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 512 (blocks: 1024)
Time taken for v2: 0.0883 ms
Final Sum = 1048576 (expected 1048576)


In [ ]:
%%writefile reduction_v3.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void reduce_v3(float *i, float *o, int n){
    extern __shared__ float sdata[];

    int tid = threadIdx.x;
    int idx  = blockIdx.x * (blockDim.x * 2) + threadIdx.x;

    float sum = 0.0f;
    if (idx < n){
        sum = i[idx];
        if (idx + blockDim.x < n){
            sum += i[idx + blockDim.x];
        }
    }
    sdata[tid] = sum;
    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride >>=1){
        if (tid < stride){
            sdata[tid] += sdata[tid + stride];
        }
        __syncthreads();
    }
    if (tid == 0){
        o[blockIdx.x] = sdata[0];
    }
}

int main(){
    int N = 1 << 20;
    size_t size = N * sizeof(float);

    const int blockSizes[] = {128, 256, 512};
    const int numTests = 3;

    float *h_a = (float *)malloc(size);
    for (int i=0; i<N; i++){
        h_a[i] = 1.0f;
    }

    float *d_a, *d_b;
    cudaMalloc((void **)&d_a, size);

    for (int t=0; t<numTests; t++){
        int threadsPerBlock = blockSizes[t];
        int blocks = (N + threadsPerBlock * 2 - 1) / (threadsPerBlock * 2);

        cudaMalloc(&d_b, blocks * sizeof(float));
        cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);

        printf("\nTesting block size: %d (blocks: %d)\n", threadsPerBlock, blocks);

        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        cudaEventRecord(start);
        reduce_v3<<<blocks, threadsPerBlock, threadsPerBlock * sizeof(float)>>>(d_a, d_b, N);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess){
            printf("CUDA Error: %s\n", cudaGetErrorString(err));
            return -1;
        }

        float gpu_time = 0;
        cudaEventElapsedTime(&gpu_time, start, stop);
        printf("Time taken for v3: %.4f ms\n", gpu_time);

        float *h_b = (float *)malloc(blocks * sizeof(float));
        cudaMemcpy(h_b, d_b, blocks * sizeof(float), cudaMemcpyDeviceToHost);

        float final_sum = 0;
        for(int i=0; i<blocks; i++){
            final_sum += h_b[i];
        }
        printf("Final Sum = %.0f (expected %d)\n", final_sum, N);

        free(h_b);
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
        cudaFree(d_b);
    }

    cudaFree(d_a);
    free(h_a);
    return 0;
}

Overwriting reduction_v3.cu


In [ ]:
!nvcc -arch=sm_75 reduction_v3.cu -o reduction_v3

In [ ]:
!./reduction_v3


Testing block size: 128 (blocks: 4096)
Time taken for v3: 0.1103 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 256 (blocks: 2048)
Time taken for v3: 0.0661 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 512 (blocks: 1024)
Time taken for v3: 0.0695 ms
Final Sum = 1048576 (expected 1048576)


In [ ]:
%%writefile reduction_v4.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void reduce_v4(float *i, float *o, int n){
    extern __shared__ float sdata[];

    int tid = threadIdx.x;
    int idx = blockIdx.x * (blockDim.x * 2) + threadIdx.x;

    float sum = 0.0f;
    if (idx < n){
        sum = i[idx];
        if (idx + blockDim.x < n){
            sum += i[idx + blockDim.x];
        }
    }
    sdata[tid] = sum;

    for (int stride = blockDim.x / 2; stride > 0; stride >>=1){
      __syncthreads();
        if (tid < stride){
            sdata[tid] += sdata[tid + stride];
        }
    }
    if (tid == 0){
        o[blockIdx.x] = sdata[0];
    }
}

int main(){
    int N = 1 << 20;
    size_t size = N * sizeof(float);

    const int blockSizes[] = {128, 256, 512};
    const int numTests = 3;

    float *h_a = (float *)malloc(size);
    for (int i=0; i<N; i++){
        h_a[i] = 1.0f;
    }

    float *d_a, *d_b;
    cudaMalloc((void **)&d_a, size);

    for (int t=0; t<numTests; t++){
        int threadsPerBlock = blockSizes[t];
        int blocks = (N + threadsPerBlock * 2 - 1) / (threadsPerBlock * 2);

        cudaMalloc(&d_b, blocks * sizeof(float));
        cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);

        printf("\nTesting block size: %d (blocks: %d)\n", threadsPerBlock, blocks);

        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        cudaEventRecord(start);
        reduce_v4<<<blocks, threadsPerBlock, threadsPerBlock * sizeof(float)>>>(d_a, d_b, N);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess){
            printf("CUDA Error: %s\n", cudaGetErrorString(err));
            return -1;
        }

        float gpu_time = 0;
        cudaEventElapsedTime(&gpu_time, start, stop);
        printf("Time taken for v4: %.4f ms\n", gpu_time);

        float *h_b = (float *)malloc(blocks * sizeof(float));
        cudaMemcpy(h_b, d_b, blocks * sizeof(float), cudaMemcpyDeviceToHost);

        float final_sum = 0;
        for(int i=0; i<blocks; i++){
            final_sum += h_b[i];
        }
        printf("Final Sum = %.0f (expected %d)\n", final_sum, N);

        free(h_b);
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
        cudaFree(d_b);
    }

    cudaFree(d_a);
    free(h_a);
    return 0;
}

Overwriting reduction_v4.cu


In [ ]:
!nvcc -arch=sm_75 reduction_v4.cu -o reduction_v4

In [ ]:
!./reduction_v4


Testing block size: 128 (blocks: 4096)
Time taken for v4: 0.1389 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 256 (blocks: 2048)
Time taken for v4: 0.0652 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 512 (blocks: 1024)
Time taken for v4: 0.0694 ms
Final Sum = 1048576 (expected 1048576)


In [ ]:
%%writefile reduce_v5.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void reduce_v5(float *i, float *o, int n){
    extern __shared__ float sdata[];
    int tid = threadIdx.x;
    int idx = blockIdx.x * (blockDim.x * 8) + threadIdx.x;

    float sum = 0.0f;
    // load 8 elements per thread
    if (idx + 7 * blockDim.x < n){
        sum += i[idx];
        sum += i[idx + blockDim.x];
        sum += i[idx + 2 * blockDim.x];
        sum += i[idx + 3 * blockDim.x];
        sum += i[idx + 4 * blockDim.x];
        sum += i[idx + 5 * blockDim.x];
        sum += i[idx + 6 * blockDim.x];
        sum += i[idx + 7 * blockDim.x];
    }
    sdata[tid] = sum;
    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 32; stride >>= 1){
        if (tid < stride){
            sdata[tid] += sdata[tid + stride];
        }
        __syncthreads();
    }
    if (tid < 32){
        volatile float *vsmem = sdata;
        vsmem[tid] += vsmem[tid + 32];
        vsmem[tid] += vsmem[tid + 16];
        vsmem[tid] += vsmem[tid + 8];
        vsmem[tid] += vsmem[tid + 4];
        vsmem[tid] += vsmem[tid + 2];
        vsmem[tid] += vsmem[tid + 1];
    }
    if (tid == 0){
        o[blockIdx.x] = sdata[0];
    }
}

int main(){
    const int N = 1 << 20;
    size_t size = N * sizeof(float);
    const int blockSizes[] = {128, 256, 512};
    int numTests = 3;

    float *h_a = (float *)malloc(size);
    for (int i=0; i<N; i++){
        h_a[i] = 1.0f;
    }

    float *d_a, *d_b;
    cudaMalloc((void **)&d_a, size);

    for (int t=0; t<numTests; t++){
        int threadsPerBlock = blockSizes[t];
        int blocks = (N + threadsPerBlock * 8 - 1) / (threadsPerBlock * 8);

        cudaMalloc(&d_b, blocks * sizeof(float));
        cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
        printf("\nTesting block size: %d (blocks: %d)\n", threadsPerBlock, blocks);

        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
        cudaEventRecord(start);
        reduce_v5<<<blocks, threadsPerBlock, threadsPerBlock * sizeof(float)>>>(d_a, d_b, N);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess){
            printf("CUDA Error: %s\n", cudaGetErrorString(err));
            return -1;
        }

        float gpu_time = 0;
        cudaEventElapsedTime(&gpu_time, start, stop);
        printf("Time taken for v5: %.4f ms\n", gpu_time);

        float *h_b = (float *)malloc(blocks * sizeof(float));
        cudaMemcpy(h_b, d_b, blocks * sizeof(float), cudaMemcpyDeviceToHost);

        float final_sum = 0;
        for(int i=0; i<blocks; i++){
            final_sum += h_b[i];
        }
        printf("Final Sum = %.0f (expected %d)\n", final_sum, N);

        free(h_b);
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
        cudaFree(d_b);
    }
    cudaFree(d_a);
    free(h_a);
    return 0;
}

Overwriting reduce_v5.cu


In [ ]:
!nvcc -arch=sm_75 reduce_v5.cu -o reduce_v5

In [ ]:
!./reduce_v5


Testing block size: 128 (blocks: 1024)
Time taken for v5: 0.1083 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 256 (blocks: 512)
Time taken for v5: 0.0241 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 512 (blocks: 256)
Time taken for v5: 0.0264 ms
Final Sum = 1048576 (expected 1048576)


In [ ]:
%%writefile reduce_v6.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void reduce_v5(float *i, float *o, int n){
    extern __shared__ float sdata[];
    int tid = threadIdx.x;
    int idx = blockIdx.x * (blockDim.x * 8) + threadIdx.x;

    float sum = 0.0f;
    // load 8 elements per thread
    if (idx + 7 * blockDim.x < n){
        sum += i[idx];
        sum += i[idx + blockDim.x];
        sum += i[idx + 2 * blockDim.x];
        sum += i[idx + 3 * blockDim.x];
        sum += i[idx + 4 * blockDim.x];
        sum += i[idx + 5 * blockDim.x];
        sum += i[idx + 6 * blockDim.x];
        sum += i[idx + 7 * blockDim.x];
    }
    sdata[tid] = sum;
    __syncthreads();

    if (blockDim.x >= 1024){
        if (tid < 512) sdata[tid] += sdata[tid + 512];
        __syncthreads();
    }
    if (blockDim.x >= 512){
        if (tid < 256) sdata[tid] += sdata[tid + 256];
        __syncthreads();
    }
    if (blockDim.x >= 256){
        if (tid < 128) sdata[tid] += sdata[tid + 128];
        __syncthreads();
    }
    if (blockDim.x >= 128){
        if (tid < 64) sdata[tid] += sdata[tid + 64];
        __syncthreads();
    }

    if (tid < 32){
        volatile float *vsmem = sdata;
        vsmem[tid] += vsmem[tid + 32];
        vsmem[tid] += vsmem[tid + 16];
        vsmem[tid] += vsmem[tid + 8];
        vsmem[tid] += vsmem[tid + 4];
        vsmem[tid] += vsmem[tid + 2];
        vsmem[tid] += vsmem[tid + 1];
    }
    if (tid == 0){
        o[blockIdx.x] = sdata[0];
    }
}

int main(){
    const int N = 1 << 20;
    size_t size = N * sizeof(float);
    const int blockSizes[] = {128, 256, 512};
    int numTests = 3;

    float *h_a = (float *)malloc(size);
    for (int i=0; i<N; i++){
        h_a[i] = 1.0f;
    }

    float *d_a, *d_b;
    cudaMalloc((void **)&d_a, size);

    for (int t=0; t<numTests; t++){
        int threadsPerBlock = blockSizes[t];
        int blocks = (N + threadsPerBlock * 8 - 1) / (threadsPerBlock * 8);

        cudaMalloc(&d_b, blocks * sizeof(float));
        cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
        printf("\nTesting block size: %d (blocks: %d)\n", threadsPerBlock, blocks);

        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
        cudaEventRecord(start);
        reduce_v5<<<blocks, threadsPerBlock, threadsPerBlock * sizeof(float)>>>(d_a, d_b, N);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess){
            printf("CUDA Error: %s\n", cudaGetErrorString(err));
            return -1;
        }

        float gpu_time = 0;
        cudaEventElapsedTime(&gpu_time, start, stop);
        printf("Time taken for v6: %.4f ms\n", gpu_time);

        float *h_b = (float *)malloc(blocks * sizeof(float));
        cudaMemcpy(h_b, d_b, blocks * sizeof(float), cudaMemcpyDeviceToHost);

        float final_sum = 0;
        for(int i=0; i<blocks; i++){
            final_sum += h_b[i];
        }
        printf("Final Sum = %.0f (expected %d)\n", final_sum, N);

        free(h_b);
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
        cudaFree(d_b);
    }
    cudaFree(d_a);
    free(h_a);
    return 0;
}

Writing reduce_v6.cu


In [ ]:
!nvcc -arch=sm_75 reduce_v6.cu -o reduce_v6

In [ ]:
!./reduce_v6


Testing block size: 128 (blocks: 1024)
Time taken for v6: 0.1069 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 256 (blocks: 512)
Time taken for v6: 0.0198 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 512 (blocks: 256)
Time taken for v6: 0.0267 ms
Final Sum = 1048576 (expected 1048576)


In [ ]:
%%writefile reduce_v7.cu
#include <stdio.h>
#include <cuda_runtime.h>

__device__ __forceinline__ float warpReduceSum(float val) {
    // __shfl_down_sync(mask, val, delta)
    // We use 0xffffffff to indicate all 32 threads in the warp are active
    // (In a real scenario with non-multiple-of-32 block sizes, we might need a tighter mask,
    // but for standard reductions blockDim is usually a multiple of 32).
    for (int offset = warpSize/2; offset > 0; offset >>= 1) {
        val += __shfl_down_sync(0xffffffff, val, offset);
    }
    return val;
}

__global__ void reduce_v7(float *i, float *o, int n) {
    extern __shared__ float sdata[];

    unsigned int tid  = threadIdx.x;
    // The Global Index assumes we are processing 8 blocks worth of data per block
    unsigned int idx  = blockIdx.x * (blockDim.x * 8) + threadIdx.x;
    // The Grid Stride moves by the total number of threads * 8
    unsigned int grid = blockDim.x * 8 * gridDim.x;

    float sum = 0.0f;

    // 1. Thread Coarsening Loop (Unrolled x8)
    while (idx < n) {
        // We must check bounds for every access because N might not be a multiple of the stride
        sum += i[idx];
        if (idx + blockDim.x < n)      sum += i[idx + blockDim.x];
        if (idx + 2 * blockDim.x < n)  sum += i[idx + 2 * blockDim.x];
        if (idx + 3 * blockDim.x < n)  sum += i[idx + 3 * blockDim.x];
        if (idx + 4 * blockDim.x < n)  sum += i[idx + 4 * blockDim.x];
        if (idx + 5 * blockDim.x < n)  sum += i[idx + 5 * blockDim.x];
        if (idx + 6 * blockDim.x < n)  sum += i[idx + 6 * blockDim.x];
        if (idx + 7 * blockDim.x < n)  sum += i[idx + 7 * blockDim.x];

        idx += grid;
    }

    sdata[tid] = sum;
    __syncthreads();

    // 2. Shared Memory Reduction
    // Reduce down to the last 32 threads (1 Warp)
    // We stop when stride is 32.
    for (int stride = blockDim.x / 2; stride >= 32; stride >>= 1) {
        if (tid < stride) {
            sdata[tid] += sdata[tid + stride];
        }
        __syncthreads();
    }

    // 3. Warp Shuffle Reduction
    // Only the first warp executes this
    if (tid < 32) {
        float val = sdata[tid];

        // Final shared mem reduction before shuffle
        // sdata[tid] currently holds partial sum. We need to add the upper half
        // if the block was larger than 64.
        // Note: The loop above stopped at stride=32, so sdata[0..31] already
        // includes the sums from sdata[32..63]. We are good to go.

        val = warpReduceSum(val);

        if (tid == 0) {
            o[blockIdx.x] = val;
        }
    }
}

int main() {
    const int N = 1 << 20; // 1M elements
    size_t size = N * sizeof(float);

    const int blockSizes[] = {128, 256, 512};
    int numTests = 3;

    float *h_a = (float *)malloc(size);
    // Initialize array to 1.0f. Expected sum = N.
    for (int i = 0; i < N; i++) h_a[i] = 1.0f;

    float *d_a, *d_b;
    cudaMalloc(&d_a, size);

    for (int t = 0; t < numTests; t++) {
        int threadsPerBlock = blockSizes[t];

        // Calculate blocks required given 8x unrolling
        int blocks = (N + threadsPerBlock * 8 - 1) / (threadsPerBlock * 8);
        // Small safeguard against empty grids
        if (blocks == 0) blocks = 1;

        cudaMalloc(&d_b, blocks * sizeof(float));
        cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);

        printf("\nTesting block size: %d (Grid size: %d)\n",
               threadsPerBlock, blocks);

        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
        cudaEventRecord(start);

        reduce_v7<<<blocks, threadsPerBlock,
                    threadsPerBlock * sizeof(float)>>>(d_a, d_b, N);

        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        float gpu_time;
        cudaEventElapsedTime(&gpu_time, start, stop);
        printf("Time taken for v7: %.4f ms\n", gpu_time);

        float *h_b = (float *)malloc(blocks * sizeof(float));
        cudaMemcpy(h_b, d_b, blocks * sizeof(float), cudaMemcpyDeviceToHost);

        float final_sum = 0;
        for (int i = 0; i < blocks; i++) final_sum += h_b[i];

        printf("Final Sum = %.0f (expected %d)\n", final_sum, N);

        free(h_b);
        cudaFree(d_b);
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    cudaFree(d_a);
    free(h_a);
    return 0;
}

Writing reduce_v7.cu


In [ ]:
!nvcc -arch=sm_75 reduce_v7.cu -o reduce_v7.cu

In [ ]:
!./reduce_v7.cu


Testing block size: 128 (Grid size: 1024)
Time taken for v7: 0.1332 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 256 (Grid size: 512)
Time taken for v7: 0.0217 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 512 (Grid size: 256)
Time taken for v7: 0.0272 ms
Final Sum = 1048576 (expected 1048576)


In [10]:
%%writefile reduce_V8.cu
#include <stdio.h>
#include <cuda_runtime.h>

__inline__ __device__ float warpReduceSum(float val){
    for (int offset = warpSize/2; offset > 0; offset >>= 1){
        val += __shfl_down_sync(0xffffffff, val, offset);
    }
    return val;
}

__global__ void reduce_v8(const float *i, float *o, int n){
    extern __shared__ float sdata[];

    // Cast input to float4 for vectorized loads
    const float4 *in_vec = reinterpret_cast<const float4*>(i);
    int num_vec = n / 4;

    int tid = threadIdx.x;
    // Each thread processes 8 floats = 2 float4 vectors per iteration
    int idx = blockIdx.x * (blockDim.x * 2) + tid;
    int stride = blockDim.x * 2 * gridDim.x;

    float sum = 0.0f;

    // Grid-stride loop: each iteration processes 2 float4 vectors (8 floats)
    while (idx < num_vec){
        // Load first float4 (4 floats)
        float4 v1 = in_vec[idx];
        sum += (v1.x + v1.y + v1.z + v1.w);

        // Load second float4 (4 more floats) if available
        if (idx + blockDim.x < num_vec){
            float4 v2 = in_vec[idx + blockDim.x];
            sum += (v2.x + v2.y + v2.z + v2.w);
        }

        idx += stride;
    }

    // Handle tail elements (if n is not divisible by 4)
    // Start from the last complete float4 position
    int tail_start = num_vec * 4;
    for (int k = tail_start + tid; k < n; k += blockDim.x){
        sum += i[k];
    }

    // Write partial sum to shared memory
    sdata[tid] = sum;
    __syncthreads();

    // Block reduction in shared memory
    for (int s = blockDim.x/2; s >= 32; s >>= 1){
        if (tid < s){
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }

    // Warp reduction
    float val = sdata[tid];
    if (tid < 32){
        val = warpReduceSum(val);
    }

    // Write block result
    if (tid == 0){
        o[blockIdx.x] = val;
    }
}

int main() {
    const int N = 1 << 20;  // 1M
    size_t size = N * sizeof(float);

    const int blockSizes[] = {128, 256, 512};
    const int numTests = 3;

    float *h_a = (float*)malloc(size);
    for (int i = 0; i < N; i++)
        h_a[i] = 1.0f;

    float *d_a;
    cudaMalloc(&d_a, size);
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);

    for (int t = 0; t < numTests; t++) {
        int threads = blockSizes[t];
        // Calculate blocks: each thread processes 8 floats = 2 float4s
        int blocks = (N + threads*8 - 1) / (threads*8);

        float *d_b;
        cudaMalloc(&d_b, blocks * sizeof(float));

        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        cudaEventRecord(start);
        reduce_v8<<<blocks, threads, threads * sizeof(float)>>>(
            d_a, d_b, N
        );
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        // Check for errors
        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess) {
            printf("CUDA Error: %s\n", cudaGetErrorString(err));
        }

        float gpu_time = 0;
        cudaEventElapsedTime(&gpu_time, start, stop);

        float *h_b = (float*)malloc(blocks * sizeof(float));
        cudaMemcpy(h_b, d_b, blocks * sizeof(float), cudaMemcpyDeviceToHost);

        float final_sum = 0;
        for (int i = 0; i < blocks; i++) final_sum += h_b[i];

        printf("\nTesting block size: %d (blocks: %d)\n", threads, blocks);
        printf("Time taken for v8 (vectorized): %.4f ms\n", gpu_time);
        printf("Final Sum = %.0f (expected %d)\n", final_sum, N);

        free(h_b);
        cudaFree(d_b);
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    cudaFree(d_a);
    free(h_a);
    return 0;
}

Overwriting reduce_V8.cu


In [11]:
!nvcc -arch=sm_75 reduce_V8.cu -o reduce_V8

In [12]:
!./reduce_V8


Testing block size: 128 (blocks: 1024)
Time taken for v8 (vectorized): 0.1268 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 256 (blocks: 512)
Time taken for v8 (vectorized): 0.0312 ms
Final Sum = 1048576 (expected 1048576)

Testing block size: 512 (blocks: 256)
Time taken for v8 (vectorized): 0.0344 ms
Final Sum = 1048576 (expected 1048576)
